# Assignment 3_2: Echo State Networks

In [100]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt
import torch
import torch.utils.data as data
import torch.nn.functional as F
import torch.optim as optim

from esn import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [101]:
narma_df = pd.read_csv('../NARMA10.csv', header=None)
narma_df.iloc[:, :20] # visualize the first 20 columns

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.083964,0.48934,0.35635,0.25024,0.23554,0.029809,0.34099,0.021216,0.035723,0.26082,0.048365,0.40907,0.40877,0.36122,0.074933,0.32980,0.25930,0.48649,0.32450,0.40017
1,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.132850,0.17536,0.37127,0.36481,0.337070,0.20447,0.33003,0.20726,0.18825,0.28343


In [102]:
x_data = torch.tensor(narma_df.iloc[0].values, dtype=torch.float32) # float 32 for better memory efficiency
y_data = torch.tensor(narma_df.iloc[1].values, dtype=torch.float32)

# 4000 tr, 1000 val, 5000 test (WARNING: load entire dataset in memory ONLY because it is small and also the NN is quite small)
dev_x, dev_y = x_data[:5000], y_data[:5000] # only used for retraining (train + val sets)

test_x, test_y = x_data[5000:], y_data[5000:]

train_x, val_x = dev_x[:4000], dev_x[4000:]
train_y, val_y = dev_y[:4000], dev_y[4000:]

In [121]:
class RegressorESN(nn.Module):
    def __init__(self, input_size:int, hidden_size:int, output_size:int, ridge_regression:float, 
                 omhega_in:float, omhega_b:float, rho:float, density:float = 1):
        
        super(RegressorESN, self).__init__()
        
        self.reservoir = Reservoir(input_size, hidden_size, omhega_in, omhega_b, rho, density)
        self.readout = Ridge(alpha=ridge_regression) # linear ridge regression of scikit-learn
        self.states = None
    
    def fit(self, input:torch.Tensor, target:torch.Tensor, washout:int = 0):
        states = self.reservoir(input, h_init=None, washout=washout)
        states = states.squeeze(1)
        self.readout.fit(states, target)

        self.states = states
        return states[-1] # return last state

    @torch.no_grad()
    def forward(self, input:torch.Tensor, h_init:torch.Tensor, washout:int = 0):
        if self.training: # avoid to recompute states
            states = self.states # states already computed during fitting
        else:
            states = self.reservoir(input, h_init=h_init, washout=washout).squeeze(1)

        return torch.from_numpy(self.readout.predict(states))

In [122]:
esn = RegressorESN(1, 3, 1, 1e-6, 0.1, 0.1, 0.9, 1)

In [106]:
train_x = train_x.unsqueeze(1).unsqueeze(1)
train_x.shape

torch.Size([4000, 1, 1])

In [110]:
l = torch.nn.L1Loss()
train_y

tensor([0.0000, 0.0000, 0.0000,  ..., 0.1929, 0.4833, 0.3957])

In [125]:
esn.fit(train_x, train_y)

train_y.unsqueeze(0)

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.1929, 0.4833, 0.3957]])